<a href="https://colab.research.google.com/github/howardatri/notes/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义 LeNet 模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 第一个卷积层，输入通道1，输出通道6，卷积核大小5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 第二个卷积层，输入通道6，输出通道16，卷积核大小5x5
        self.conv2 = nn.Conv2d(6, 16, 5)

        # 为了计算第一个全连接层的输入尺寸，我们通过一个 dummy tensor 来确定
        # 输入图像尺寸为 28x28
        dummy_input = torch.randn(1, 1, 28, 28)
        x = F.max_pool2d(F.relu(self.conv1(dummy_input)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        self._num_flat_features = x.size(1) * x.size(2) * x.size(3)

        # 第一个全连接层
        self.fc1 = nn.Linear(self._num_flat_features, 120)
        # 第二个全连接层，输入120，输出84
        self.fc2 = nn.Linear(120, 84)
        # 第三个全连接层，输入84，输出10（对应10个类别）
        self.fc3 = nn.Linear(84, 10)


    def forward(self, x):
        # 第一个卷积 -> ReLu -> 最大池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 第二个卷积 -> ReLu -> 最大池化
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 展平操作，将多维张量展平为一维
        x = x.view(-1, self._num_flat_features)
        # 第一个全连接 -> ReLu
        x = F.relu(self.fc1(x))
        # 第二个全连接 -> ReLu
        x = F.relu(self.fc2(x))
        # 第三个全连接（输出层）
        x = self.fc3(x)
        return x

# 实例化 LeNet 模型
# model = LeNet()
# print(model)

In [ ]:
import torchvision
import torchvision.transforms as transforms

# 定义数据转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为 Tensor
    transforms.Normalize((0.5,), (0.5,)) # 标准化
])

# 下载并加载训练集
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# 下载并加载测试集
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# 实例化 LeNet 模型
model = LeNet()

# 测试模型（使用一个批次的数据）
dataiter = iter(testloader)
images, labels = next(dataiter)

# 将数据通过模型
outputs = model(images)

print("模型输出的形状:", outputs.shape)

模型输出的形状: torch.Size([4, 10])
